一、如果您在完全相同的训练数据上训练了五个不同的模型，并且它们都达到了95％的精度，那么您是否有可能将这些模型结合起来以获得更好的结果？ 如果是这样，怎么办？ 如果没有，为什么？

如果您已经训练了五个不同的模型，并且它们都达到了95％的精度，则可以尝试将它们组合成一个投票集成器，这通常会给您带来更好的结果。如果模型差异很大（例如SVM分类器，决策树分类器，逻辑回归分类器等），则效果更好。如果在不同的训练实例上进行训练（这是bagging和pasting集成的重点），那就更好了，但是如果没有，只要模型非常不同，这仍然有效。

二、硬投票和软投票分类器有什么区别？

硬投票分类器仅计算集合中每个分类器的投票，然后选择获得最多投票的类别。软投票分类器计算每个类别的平均估计类别概率，并选择概率最高的类别。 这会给高置信度的投票赋予更大的权重，并且通常会表现更好，但是仅当每个分类器都能够估计分类概率时才有效（例如，对于Scikit-Learn中的SVM分类器，您必须设置probability= True）

三、是否可以通过在多台服务器之间分配bagging来加快训练的速度？ pasting集成，boosting集成，随机森林或堆栈集成呢？

通过将bagging集成分布到多个服务器上来加快训练的速度，这是很有可能的，因为集成中的每个预测变量都相互独立。出于相同的原因，pasting集成和随机森林也是如此。但是，boosting集成中的每个预测变量都是基于先前的预测变量构建的，因此训练必须是顺序的，并且通过在多个服务器之间分布训练将不会获得任何收益。关于堆栈集成，给定层中的所有预测变量彼此独立，因此可以在多台服务器上并行训练它们。但是，只有在对上一层中的预测变量进行了全部训练之后，才能对一层中的预测变量进行训练。

四、out-of-bag评估的优点：

通过out-of-bag，可以使用未经过训练（被排除）的实例评估bagging组合中的每个预测变量。这使得可以对集合进行相当公正的评估，而无需其他验证集。因此，您有更多可用于训练的实例，并且您的集成结果可以稍微好一些。

五、是什么让Extra-trees比常规的随机森林更随机？ 这种额外的随机性如何提供帮助？ Extra-trees是比常规随机森林慢还是快？

在“随机森林”中生长一棵树时，仅考虑特征的随机子集在每个节点处进行拆分。对于Extra-Trees也是如此，但它们又走了一步：与其像常规的决策树一样搜索最佳阈值，不如对每个特征使用随机阈值。这种额外的随机性就像正规化的形式：如果“随机森林”拟合训练数据，则“额外树”可能会表现更好。此外，由于Extra-Trees不会搜索最佳阈值，因此它们的训练速度比Random Forests要快得多。但是，进行预测时，它们既不比随机森林快也不慢。

六、如果AdaBoost集合不能拟合训练数据，则应调整哪些超参数以及如何调整？

如果AdaBoost集合不能拟合训练数据，则可以尝试增加n_estimators或减少基本估计量的正则化超参数。您也可以尝试稍微提高学习速度。

七、如果Gradient Boosting集成对训练集过拟合，您应该增加还是降低学习率？

如果Gradient Boosting集成对训练集过拟合，则应尝试降低学习率。您还可以使用提前停止来找到正确数量的n_estimators（可能设置太多）

八、加载MNIST数据，并将其分为训练集、验证集和测试集（例如，使用1100个实例进行训练，使用300个进行验证，并使用397个进行测试）。然后训练各种分类器，例如随机森林分类器，extra-tree分类器和SVM分类器。接下来，尝试使用软投票或硬投票将它们组合成一个在验证集上胜过每个单独分类器的集合。找到一个后，在测试集上尝试一下。与单个分类器相比，它的性能要好多少？

In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

1. 加载数据集

In [3]:
digits = load_digits()

In [4]:
data, labels = digits.data, digits.target

In [5]:
data.shape

(1797, 64)

发现只有1797条数据，每个手写数字图片由8*8个像素点组成

2. 划分数据集，划分为三部分（训练集、验证集和测试集）

首先划分训练验证集和测试集

In [7]:
X_train_val, X_test, y_train_val, y_test = train_test_split(data, labels, test_size=397, random_state=42)

将训练验证集划分为训练集和验证集

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=300, random_state=42)

3. 然后训练各种分类器，例如随机森林分类器，extra-tree分类器和SVM

In [9]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

D:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [10]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [11]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
Training the ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=42, verbose=0,
                     warm_start=False)
Training 

D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Training the MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=42, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)


4. 查看它们在验证集上的效果

In [12]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.9566666666666667,
 0.9733333333333334,
 0.9266666666666666,
 0.9666666666666667]

发现目前最好的是extra-tree分类器

5. 接下来，尝试使用软投票或硬投票分类器将它们组合成一个在验证集上胜过它们的集成分类器。

In [13]:
from sklearn.ensemble import VotingClassifier

In [14]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

In [15]:
voting_clf = VotingClassifier(named_estimators)

In [16]:
voting_clf.fit(X_train, y_train)

D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                                     n_jobs=N

In [17]:
voting_clf.score(X_val, y_val)

0.9733333333333334

In [18]:
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

[0.9566666666666667,
 0.9733333333333334,
 0.9266666666666666,
 0.9666666666666667]

让我们删除SVM以查看性能是否有所提高。可以使用set_params（）将估算器设置为None来删除估算器，如下所示：

In [19]:
voting_clf.set_params(svm_clf=None)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                                     n_jobs=N

In [20]:
voting_clf.estimators

[('random_forest_clf',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                         max_depth=None, max_features='auto', max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, n_estimators=100,
                         n_jobs=None, oob_score=False, random_state=42, verbose=0,
                         warm_start=False)),
 ('extra_trees_clf',
  ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
  

In [21]:
voting_clf.estimators_

[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=None, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=42, verbose=0,
                        warm_start=False),
 ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False),
 LinearSVC(C=1.0, c

删除svm训练器

In [22]:
del voting_clf.estimators_[2]

In [23]:
voting_clf.score(X_val, y_val)

0.9666666666666667

In [24]:
voting_clf.voting = "soft"

In [25]:
voting_clf.score(X_val, y_val)

0.9733333333333334

很显然，软投票好一些

In [26]:
voting_clf.score(X_test, y_test)

0.9798488664987406

In [27]:
[estimator.score(X_test, y_test) for estimator in voting_clf.estimators_]

[0.9798488664987406, 0.9798488664987406, 0.9773299748110831]

九、运行上一个练习中的各个分类器，以对验证集进行预测，并使用结果预测创建一个新的训练集：每个训练实例都是一个向量，其中包含来自所有分类器的图像预测集，目标是图片的类别。在这个新的训练集上训练一个分类器，恭喜，您刚刚训练了一个blender，并且与分类器一起构成了一个堆叠合奏！ 现在评估测试集上的整体。 对于测试集中的每幅图像，请使用所有分类器进行预测，然后将预测结果输入到blender中，以获取整体的预测结果。与您之前训练的投票分类器相比如何？

In [29]:
import numpy as np

1. 生成预测结果的训练集

In [30]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [31]:
X_val_predictions

array([[1., 1., 1., 1.],
       [9., 9., 9., 9.],
       [3., 3., 3., 3.],
       ...,
       [7., 7., 7., 7.],
       [1., 1., 1., 1.],
       [0., 0., 0., 0.]], dtype=float32)

2. 创建一个random_forestblender分类器

In [32]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [33]:
rnd_forest_blender.oob_score_

0.97

生成预测训练集

In [34]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [35]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
accuracy_score(y_test, y_pred)

0.9773299748110831

可以发现，stacking与软投票的效果差不多，都略优于硬投票机制